In [1]:
import os
import pickle

import numpy as np
import numpy.ma as ma

from hbmep.nn import functional as F
from hbmep.model.utils import Site as site

from models import (
    NonHierarchicalBayesianModel
)

In [2]:
src = "/home/andres/repos/rat-mapping-paper/reports/C_SMA_LAR/non_hierarchical_bayesian_model/processed_inference.pkl"
with open(src, "rb") as f:
    (
        df,
        encoder_dict,
        model,
        posterior_samples,
        y,
        subjects_inv,
        compound_positions_inv,
        compound_size_inv,
    ) = pickle.load(f)


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [3]:
a = posterior_samples[site.a]
a.shape


(400, 8, 70, 3, 6)

In [4]:
y.shape # samples, subjects, positions, sizes, muscles, x) 

(400, 8, 70, 3, 6, 1000)

In [21]:
df[["channel1_segment", "channel2_segment"]].apply(tuple, axis=1).unique()


array([('C5', 'C5'), (nan, 'C5'), ('C6', 'C6'), ('C5', 'C6'), (nan, 'C6'),
       ('C7', 'C7'), (nan, 'C7')], dtype=object)

In [26]:
df[["channel1_laterality", "channel2_laterality"]].apply(tuple, axis=1)


0          (M, L)
1          (M, L)
2          (M, L)
3          (M, L)
4          (M, L)
           ...   
20803    (nan, L)
20804    (nan, M)
20805      (L, L)
20806    (nan, L)
20807      (M, L)
Length: 20808, dtype: object

In [32]:
df.columns


Index(['pulse_amplitude', 'pulse_train_frequency', 'pulse_period',
       'pulse_duration', 'pulse_count', 'train_delay', 'channel1_1',
       'channel1_2', 'channel1_3', 'channel1_4', 'channel2_1', 'channel2_2',
       'channel2_3', 'channel2_4', 'charge_params_1', 'charge_params_2',
       'charge_params_3', 'charge_params_4', 'bank_check', 'run', 'enabled',
       'channel_failA', 'channel_failB', 'channel_fail_comb', 'time_pulse',
       'time', 'ix_onsets', 'LBiceps', 'LFCR', 'LECR', 'LTriceps', 'LADM',
       'LDeltoid', 'LBicepsFemoris', 'RBiceps', 'channel1_laterality',
       'channel1_segment', 'channel1_designation', 'channel1_size',
       'channel2_laterality', 'channel2_segment', 'channel2_designation',
       'channel2_size', 'compound_position', 'compound_charge_params',
       'compound_size', 'participant', 'subdir_pattern', 'charge_param_error'],
      dtype='object')

In [35]:
print(encoder_dict[model.features[1]].inverse_transform(df.compound_position.values).tolist())

['C5M-C5LL', 'C5M-C5LL', 'C5M-C5LL', 'C5M-C5LL', 'C5M-C5LL', 'C5M-C5LL', 'C5M-C5LL', 'C5M-C5LL', 'C5M-C5LL', 'C5M-C5LL', 'C5M-C5LL', 'C5M-C5LL', 'C5M-C5LL', 'C5M-C5LL', 'C5M-C5LL', 'C5M-C5LL', 'C5M-C5LL', 'C5M-C5LL', 'C5M-C5LL', 'C5M-C5LL', '-C5LL', 'C5M-C5LM', '-C5M', 'C5M-C5LL', '-C5LM', '-C5L', 'C5M-C5L', 'C5M-C5LL', '-C5M', 'C5M-C5L', '-C5LM', '-C5LL', 'C5M-C5LM', '-C5L', '-C5LM', 'C5M-C5L', 'C5M-C5LM', 'C5M-C5LL', '-C5L', '-C5M', '-C5LL', '-C5M', 'C5M-C5L', '-C5LL', '-C5LM', 'C5M-C5LL', 'C5M-C5LM', '-C5L', 'C5M-C5LM', 'C5M-C5L', '-C5LM', '-C5L', 'C5M-C5LL', '-C5LL', '-C5M', 'C5M-C5LL', '-C5L', 'C5M-C5LM', '-C5LL', 'C5M-C5L', '-C5M', '-C5LM', 'C5M-C5LM', 'C5M-C5L', '-C5L', '-C5M', '-C5LM', 'C5M-C5LL', '-C5LL', 'C5M-C5LM', '-C5M', '-C5LL', '-C5L', 'C5M-C5LL', '-C5LM', 'C5M-C5L', 'C5M-C5L', '-C5L', '-C5M', 'C5M-C5LM', '-C5LL', '-C5LM', 'C5M-C5LL', 'C5M-C5L', 'C5M-C5LL', '-C5M', '-C5LL', '-C5L', 'C5M-C5LM', '-C5LM', 'C5M-C5LL', '-C5L', '-C5LM', 'C5M-C5LM', '-C5M', '-C5LL', 'C5M-C5L', 

In [41]:
(df.channel2_laterality == "LL").unique()

array([False])

In [31]:
df.channel1_laterality.unique()

array(['M', nan, 'L', 'R'], dtype=object)

In [24]:
compound_positions_inv[21]

'C5LM1-C5L'

In [25]:
ind = df[["channel1_laterality", "channel2_laterality"]].apply(tuple, axis=1).isin([('L', 'L')])

df[ind].compound_position

395      21
400      23
404      27
406      22
409      28
         ..
20795    47
20798    44
20800    36
20802    45
20805    46
Name: compound_position, Length: 4889, dtype: int64

In [15]:
compound_size_inv

['B', 'S', 'S-B']

In [ ]:
degrees = ["LL", "L", "LM", "LM1", "LM2", "M"]
positions = ["C5", "C6", "C7"]
polarities = ["monopolar", "bipolar"]

for polarity_ind, polarity in enumerate(polarities):
    for size_ind, size in enumerate(compound_size_inv):
        for lat_ind, lat in enumerate(degrees):
            for pos_ind, pos in enumerate(positions):
                # Make the combination
                if polarity == "monopolar":
                    

In [14]:
df[["channel1_segment", "channel2_segment"]].apply(tuple, axis=1).unique()


array([('C5', 'C5'), (nan, 'C5'), ('C6', 'C6'), ('C5', 'C6'), (nan, 'C6'),
       ('C7', 'C7'), (nan, 'C7')], dtype=object)

In [13]:
df.channel1_segment.unique()

array(['C5', nan, 'C6', 'C7'], dtype=object)

In [12]:
df.channel2_segment.unique()

array(['C5', 'C6', 'C7'], dtype=object)

In [7]:
sorted(compound_positions_inv)


['-C5L',
 '-C5LL',
 '-C5LM',
 '-C5LM1',
 '-C5LM2',
 '-C5M',
 '-C6L',
 '-C6LL',
 '-C6LM',
 '-C6LM1',
 '-C6LM2',
 '-C6M',
 '-C6R',
 '-C6RM',
 '-C6RR',
 '-C7LM',
 '-C7M',
 '-C7R',
 '-C7RM',
 '-C7RR',
 'C5L-C5LL',
 'C5LM1-C5L',
 'C5LM1-C5LL',
 'C5LM2-C5L',
 'C5LM2-C5LL',
 'C5LM2-C5LM1',
 'C5LM2-C5M',
 'C5LM2-C6L',
 'C5LM2-C6LL',
 'C5M-C5L',
 'C5M-C5LL',
 'C5M-C5LM',
 'C5M-C5LM1',
 'C5M-C5LM2',
 'C5M-C6L',
 'C5M-C6LL',
 'C6L-C6LL',
 'C6LL-C6L',
 'C6LM-C6L',
 'C6LM-C6LL',
 'C6LM-C6M',
 'C6LM-C6R',
 'C6LM-C6RM',
 'C6LM-C6RR',
 'C6LM1-C6L',
 'C6LM1-C6LL',
 'C6LM2-C6L',
 'C6LM2-C6LL',
 'C6LM2-C6LM1',
 'C6M-C6L',
 'C6M-C6LL',
 'C6M-C6LM',
 'C6M-C6LM1',
 'C6M-C6LM2',
 'C6M-C6R',
 'C6M-C6RM',
 'C6M-C6RR',
 'C6R-C6RR',
 'C6RM-C6R',
 'C6RM-C6RR',
 'C7LM-C7M',
 'C7LM-C7R',
 'C7LM-C7RM',
 'C7LM-C7RR',
 'C7M-C7R',
 'C7M-C7RM',
 'C7M-C7RR',
 'C7R-C7RR',
 'C7RM-C7R',
 'C7RM-C7RR']

In [7]:
y_swap = ma.swapaxes(y, 0, 1)
y_swap = ma.swapaxes(y_swap, 1, 2)
y_swap.shape

(8, 70, 400, 3, 6, 1000)

In [7]:
t = list(zip(subjects_inv, compound_positions_inv, compound_size_inv))


In [4]:
compound_positions_inv = np.array(compound_positions_inv)

In [8]:
y_big = y_swap[...,0,:,:]
y_small = y_swap[...,1,:,:]

In [ ]:
t = [
    val.split("-") for val in check if val.split("-")[0]
]
t = [
    (x[2:], y[2:]) for x, y in t
]
t = [x + "-" + y for x, y in t]


In [ ]:
for lat in laterality:
    for l in level:
        

In [9]:
laterality = ["M", "LM", "LM1", "LM2", "L", "LL"]
level = ["-C5", "-C6", "-C7", "C5-C5", "C6-C6", "C7-C7", "C5-C6"]
ground_positions = ["-C5", "-C6", "-C7"]
single_positions = ["C5-C5", "C6-C6", "C7-C7"]
two_level = ["C5-C6"]

In [10]:
#400, 1000
y_big.shape
y_big_new = []
mask_big = []

for i, sub in enumerate(subjects_inv):
    for l in laterality:
        for r_lat in laterality:
            for v in level:
                if v in ground_positions:
                    temp_pos = v+l
                elif v in single_positions:
                    left = v.split("-")[0]
                    right = v.split("-")[1]
                    for l2 in laterality:
                        temp_pos = left+l+"-"+right+r_lat
                elif v in two_level:
                    left = v.split("-")[0]
                    right = v.split("-")[1]
                    for l2 in laterality:
                        temp_pos = left+l+"-"+right+r_lat
                        
                pos_ind = np.where(compound_positions_inv == temp_pos)
                arr_temp = y_big[i, pos_ind[0], ...]
                
                if len(pos_ind[0]) > 0:
                    y_big_new.append(arr_temp)
                    mask_big.append(np.full((1, 400, 6, 1000), False))
                else:
                    y_big_new.append(np.zeros((1, 400, 6, 1000)))
                    mask_big.append(np.full((1, 400, 6, 1000), True))
                
y_big_new = np.array(y_big_new)     
y_big_new = y_big_new.reshape(len(subjects_inv), len(laterality), len(laterality), len(level), 400, 6, 1000)  

In [31]:
y_big_new.shape

AttributeError: 'list' object has no attribute 'shape'

In [18]:
y_small_new = []
mask_small = []
for i, sub in enumerate(subjects_inv):
    for l in laterality:
        for v in level:
            if v in ground_positions:
                temp_pos = v+l
            elif v in single_positions:
                left = v.split("-")[0]
                right = v.split("-")[1]
                for l2 in laterality:
                    temp_pos = left+l+"-"+right+l2
            elif v in two_level:
                left = v.split("-")[0]
                right = v.split("-")[1]
                for l2 in laterality:
                    temp_pos = left+l+"-"+right+l2
            pos_ind = np.where(compound_positions_inv == temp_pos)
            arr_temp = y_small[i, pos_ind[0], ...]
            
            if len(pos_ind[0]) > 0:
                y_small_new.append(arr_temp)
                mask_small.append(np.full((1, 400, 6, 1000), False))
            else:
                y_small_new.append(np.zeros((1, 400, 6, 1000)))
                mask_small.append(np.full((1, 400, 6, 1000), True))
                
y_small_new = np.array(y_small_new)     
y_small_new = y_small_new.reshape(len(subjects_inv), len(laterality), len(level), 400, 6, 1000)  

In [19]:
def get_entropy(arr):
    y = arr[:, :, ...]
    x = arr[:, :,:, 0, 0, :]

    y_max = ma.max(y, axis=(1,2,-1), keepdims=True) 

    y = ma.where(y, y / y_max, 0.)

    p = ma.sum(y, axis=-2, keepdims=True)
    p = ma.where(y, y / p, 0.)

    plogp = ma.where(p, p * ma.log(p), 0.)

    entropy = ma.where(
        ma.any(p, axis=-2, keepdims=True),
        (
            1
            + (ma.sum(plogp, axis=-2, keepdims=True) / np.log(plogp.shape[-2]))
        ),
        0.
    )
    entropy = entropy[..., 0, :]

    return entropy, x


In [20]:
entropy_big, x_big = get_entropy(y_big_new)

In [ ]:
area_list = []
for subject_ind, subject in enumerate(subjects_inv):

    for cpos_ind in range(len(entropy_big[0,:,0,0])):
        cpos = compound_positions_inv[configuration[cpos_ind]]
        x_temp = x[subject_ind, 0, ...].tolist()
        
        sample_integrals = entropy[subject_ind, cpos_ind, :, :]
        sample_integrals_mean = sample_integrals.mean(axis=0)
        
        area = np.trapz(sample_integrals_mean, x_temp)
        area_list.append((subject, cpos, area))

filt_positions = list(set((a[1] for a in area_list)))

In [ ]:
entropy_small, x_small = get_entropy(y_small_new)